<a href="https://colab.research.google.com/github/aderdouri/ql_web_app/blob/master/ql_notebooks/everestoption.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import QuantLib as ql
import unittest
import math

# Helper utilities (assuming flat_rate_py, flat_vol_py are defined as in previous examples)
def flat_rate_py(evaluation_date_or_value, day_counter_or_rateval, day_counter_if_date=None):
    if isinstance(evaluation_date_or_value, ql.Date):
        evaluation_date = evaluation_date_or_value
        rate_val = day_counter_or_rateval
        dc = day_counter_if_date
    else:
        evaluation_date = ql.Settings.instance().evaluationDate
        rate_val = evaluation_date_or_value
        dc = day_counter_or_rateval

    if isinstance(rate_val, ql.Quote):
        quote_handle = ql.QuoteHandle(rate_val)
    elif isinstance(rate_val, float):
        quote_handle = ql.QuoteHandle(ql.SimpleQuote(rate_val))
    else:
        quote_handle = rate_val
    return ql.FlatForward(evaluation_date, quote_handle, dc)

def flat_vol_py(evaluation_date_or_value, day_counter_or_volval, day_counter_if_date=None):
    if isinstance(evaluation_date_or_value, ql.Date):
        evaluation_date = evaluation_date_or_value
        vol_val = day_counter_or_volval
        dc = day_counter_if_date
    else:
        evaluation_date = ql.Settings.instance().evaluationDate
        vol_val = evaluation_date_or_value
        dc = day_counter_or_volval

    if isinstance(vol_val, ql.Quote):
        vol_quote_handle = ql.QuoteHandle(vol_val)
    elif isinstance(vol_val, float):
        vol_quote_handle = ql.QuoteHandle(ql.SimpleQuote(vol_val))
    else:
        vol_quote_handle = vol_val
    return ql.BlackConstantVol(evaluation_date, ql.NullCalendar(), vol_quote_handle, dc)

class EverestOptionTests(unittest.TestCase):

    def setUp(self):
        self.saved_eval_date = ql.Settings.instance().evaluationDate
        # Set a default evaluation date for tests
        self.today = ql.Date(15, ql.May, 2007) # Arbitrary fixed date
        ql.Settings.instance().evaluationDate = self.today

    def tearDown(self):
        ql.Settings.instance().evaluationDate = self.saved_eval_date

    def test_cached_value(self):
        print("Testing Everest option against cached values...")

        current_eval_date = ql.Settings.instance().evaluationDate # Use the one from setUp

        dc = ql.Actual360()
        exercise_date = current_eval_date + ql.Period(360, ql.Days)
        exercise = ql.EuropeanExercise(exercise_date)

        notional = 1.0
        guarantee = 0.0
        option = ql.EverestOption(notional, guarantee, exercise)

        risk_free_rate_handle = ql.YieldTermStructureHandle(
            flat_rate_py(current_eval_date, 0.05, dc)
        )

        processes_list = [] # Python list to hold StochasticProcess1D objects
        dummy_underlying_quote = ql.QuoteHandle(ql.SimpleQuote(1.0))

        # Process 0
        processes_list.append(ql.BlackScholesMertonProcess(
            dummy_underlying_quote,
            ql.YieldTermStructureHandle(flat_rate_py(current_eval_date, 0.01, dc)),
            risk_free_rate_handle,
            ql.BlackVolTermStructureHandle(flat_vol_py(current_eval_date, 0.30, dc))
        ))
        # Process 1
        processes_list.append(ql.BlackScholesMertonProcess(
            dummy_underlying_quote,
            ql.YieldTermStructureHandle(flat_rate_py(current_eval_date, 0.05, dc)),
            risk_free_rate_handle,
            ql.BlackVolTermStructureHandle(flat_vol_py(current_eval_date, 0.35, dc))
        ))
        # Process 2
        processes_list.append(ql.BlackScholesMertonProcess(
            dummy_underlying_quote,
            ql.YieldTermStructureHandle(flat_rate_py(current_eval_date, 0.04, dc)),
            risk_free_rate_handle,
            ql.BlackVolTermStructureHandle(flat_vol_py(current_eval_date, 0.25, dc))
        ))
        # Process 3
        processes_list.append(ql.BlackScholesMertonProcess(
            dummy_underlying_quote,
            ql.YieldTermStructureHandle(flat_rate_py(current_eval_date, 0.03, dc)),
            risk_free_rate_handle,
            ql.BlackVolTermStructureHandle(flat_vol_py(current_eval_date, 0.20, dc))
        ))

        # Convert Python list of processes to ql.StochasticProcess1DVector
        # processes_vector = ql.StochasticProcess1DVector()
        # for p in processes_list:
        #     processes_vector.append(p)
        # Simpler: directly pass list if SWIG handles it
        processes_vector = processes_list


        correlation_matrix = ql.Matrix(4, 4)
        correlation_matrix[0,0]=1.00; correlation_matrix[0,1]=0.50; correlation_matrix[0,2]=0.30; correlation_matrix[0,3]=0.10
        correlation_matrix[1,0]=0.50; correlation_matrix[1,1]=1.00; correlation_matrix[1,2]=0.20; correlation_matrix[1,3]=0.40
        correlation_matrix[2,0]=0.30; correlation_matrix[2,1]=0.20; correlation_matrix[2,2]=1.00; correlation_matrix[2,3]=0.60
        correlation_matrix[3,0]=0.10; correlation_matrix[3,1]=0.40; correlation_matrix[3,2]=0.60; correlation_matrix[3,3]=1.00

        stochastic_process_array = ql.StochasticProcessArray(processes_vector, correlation_matrix)

        seed = 86421
        fixed_samples = 1023
        minimum_tol_for_accuracy_test = 1.0e-2 # from C++ test (minimumTol * value, assuming value ~1)

        # MCEverestEngine with fixed samples
        # C++: MakeMCEverestEngine<PseudoRandom>(process).withStepsPerYear(1)...
        # Python: ql.MCEverestEngine(process, rng="PseudoRandom", ...). The rng might be default or specified.
        # The setters in Python are usually direct method calls.
        engine1 = ql.MCEverestEngine(stochastic_process_array)
        engine1.withStepsPerYear(1)
        engine1.withSamples(fixed_samples)
        engine1.withSeed(seed)
        # engine1.withMcEngine("PseudoRandom") # If RNG choice is via string
        # Or the constructor might take RNG traits directly.
        # Default RNG for MCEverestEngine in Python bindings is usually MersenneTwister.
        # If "PseudoRandom" specifically refers to that or another basic RNG, it should be fine.

        option.setPricingEngine(engine1)
        value1 = option.NPV()
        stored_value = 0.75784944
        tolerance_cached = 1.0e-8

        self.assertAlmostEqual(value1, stored_value, delta=tolerance_cached,
                               msg=f"Cached value mismatch: Calculated {value1:.8f}, Expected {stored_value:.8f}")

        # MCEverestEngine with absolute tolerance
        # tolerance_for_engine = option.errorEstimate() / 2.0 # errorEstimate from previous run
        # tolerance_for_engine = min(tolerance_for_engine, minimum_tol_for_accuracy_test * value1)
        # The C++ divides by 2.0 which is aggressive for MC.
        # Let's use a reasonable tolerance based on the expected value for the accuracy test.
        # Often the errorEstimate itself is what you'd compare against a target.
        # The C++ test sets the engine's tolerance to half the previous error estimate or a min threshold.

        # Target tolerance for the engine to achieve
        target_accuracy_for_engine = min(option.errorEstimate() / 2.0, minimum_tol_for_accuracy_test)
        # Ensure it's positive
        target_accuracy_for_engine = max(target_accuracy_for_engine, 1e-5) # Safety for very small error estimates

        engine2 = ql.MCEverestEngine(stochastic_process_array)
        engine2.withStepsPerYear(1)
        engine2.withAbsoluteTolerance(target_accuracy_for_engine)
        engine2.withSeed(seed)
        # If withSamples(0) or similar is needed to trigger tolerance-based run:
        # engine2.withSamples(0) # or withMaxSamples(some_large_number)

        option.setPricingEngine(engine2)
        option.NPV() # Trigger calculation
        reached_accuracy = option.errorEstimate()

        # The test is that the *reached* accuracy is *better* than the target set for the engine.
        self.assertLessEqual(reached_accuracy, target_accuracy_for_engine,
                              msg=f"Accuracy mismatch: Reached {reached_accuracy:.2e}, Expected < {target_accuracy_for_engine:.2e}")


if __name__ == '__main__':
    print("Testing QuantLib " + ql.__version__)
    unittest.main(argv=['first-arg-is-ignored'], exit=False)